In [2]:
import asyncio
import nest_asyncio
from typing import Dict

import os, sys
home_path = os.path.expanduser("~")
sys.path.append(os.path.join(home_path, "github", "Thunder", "Binance"))

nest_asyncio.apply()

import Workspace.Utils.TradingUtils as tr_utils
import Workspace.Utils.BaseUtils as base_utils

from SystemTrading.MarketDataFeed.Private.PrivateFetcher import PrivateFetcher

queues = []
for _ in range(3):
    queues.append(asyncio.Queue())
queue = tuple(queues)

events = []
for _ in range(2):
    events.append(asyncio.Event())
events = tuple(events)

instance = PrivateFetcher(*queues, *events)
balance, order = asyncio.run(instance.debug())

In [3]:
a, b, c = order

In [5]:
base_utils.convert_dict_to_literal(a)

{'orderId': 626454372802,
 'symbol': 'BTCUSDT',
 'status': 'NEW',
 'clientOrderId': 'electron_nN68IuWE3b5qdNaHnbxF',
 'price': 0,
 'avgPrice': 0,
 'origQty': 0,
 'executedQty': 0,
 'cumQuote': 0.0,
 'timeInForce': 'GTE_GTC',
 'type': 'TAKE_PROFIT_MARKET',
 'reduceOnly': True,
 'closePosition': True,
 'side': 'BUY',
 'positionSide': 'BOTH',
 'stopPrice': 75795.2,
 'workingType': 'MARK_PRICE',
 'priceProtect': True,
 'origType': 'TAKE_PROFIT_MARKET',
 'priceMatch': None,
 'selfTradePreventionMode': 'EXPIRE_MAKER',
 'goodTillDate': 0,
 'time': 1742216255997,
 'updateTime': 1742216255997}

In [ ]:
from SystemConfig import Streaming

symbols = Streaming.symbols
order_types = ("STOP_MARKET", "TAKE_PROFIT_MARKET", "LIMIT")
data_keys = ('orderId', 'status', 'clientOrderId', 'price', 'avgPrice', 'origQty', 'executedQty', 'cumQuote',
             'timeInForce','type', 'reduceOnly', 'closePosition', 'side', 'positionSide', 'stopPrice', 'workingType',
             'priceProtect', 'origType', 'priceMatch', 'selfTradePreventionMode', 'goodTillDate', 'time', 'updateTime')

class OrderStatus:
    __slots__ = tuple(symbols)
    def __init__(self):
        self.clear()
    
    def clear(self):
        for attr in self.__slots__:
            setattr(self, attr, SymbolStorage())

    def set_data(self, data:Dict):
        symbol = data["symbol"]
        getattr(self, symbol).set_data(data)
    
    def get_data(self, symbol:str, order_type:str):
        symbol_data = getattr(self, symbol)
        if hasattr(symbol_data, order_type):
            return symbol_data.get_data(order_type)
        else:
            return None
    
    def get_fields(self):
        return list(self.__slots__)
    
    def to_dict(self):
        result = {}
        for attr in self.__slots__:
            data = getattr(self, attr)
            if isinstance(data, list):
                continue
            result[attr] = getattr(self, attr).to_dict()
        return result

    def __repr__(self):
        return str(self.to_dict())

class SymbolStorage:
    """
    중간 위치 저장 storage다.

    Returns:
        _type_: _description_
    """
    __slots__ = order_types
    def __init__(self):
        for attr in self.__slots__:
            setattr(self, attr, [])
    
    def set_data(self, data):
        order_type = data['type']
        setattr(self, order_type, TypeStoreage())
        getattr(self, order_type).set_data(**data)
    
    def get_data(self, order_type:str):
        return getattr(self, order_type)
    
    def get_data_detail(self, order_type:str, last_key:str):
        return getattr(self, order_type).get_data(last_key)
    
    def get_fields(self):
        return list(self.__slots__)
    
    def to_dict(self):
        result = {}
        for attr in self.__slots__:
            if isinstance(getattr(self, attr), list):
                continue
            result[attr] = getattr(self, attr).to_dict()
        return result

class TypeStoreage:
    """
    최하위 저장 storage

    Returns:
        _type_: _description_
    """
    __slots__ = data_keys
    def __init__(self):
        for attr in self.__slots__:
            setattr(self, attr, None)
    
    def set_data(self, **kwargs):
        for k, v in kwargs.items():
            if k in self.__slots__:  # 유효한 필드만 설정
                setattr(self, k, v)
    
    def get_data(self, field:str):
        return getattr(self, field)
    
    def get_fields(self):
        return list(self.__slots__)
    
    def to_dict(self):
        result = {}
        for attr in self.__slots__:
            result[attr] = getattr(self, attr)
        return result

In [7]:
dummy_ = OrderStatus()
dummy_.set_data(a)
dummy_.set_data(b)
dummy_.set_data(c)

In [8]:
from pprint import pprint
data = dummy_.to_dict()

In [9]:
pprint(data)

{'BNBUSDT': {},
 'BTCUSDT': {'LIMIT': {'avgPrice': '0',
                       'clientOrderId': 'ios_HdPgrWkOA4wtWUAaTR1H',
                       'closePosition': False,
                       'cumQuote': '0.00000',
                       'executedQty': '0',
                       'goodTillDate': 0,
                       'orderId': 627105710032,
                       'origQty': '0.004',
                       'origType': 'LIMIT',
                       'positionSide': 'BOTH',
                       'price': '87605.10',
                       'priceMatch': 'NONE',
                       'priceProtect': False,
                       'reduceOnly': False,
                       'selfTradePreventionMode': 'EXPIRE_MAKER',
                       'side': 'SELL',
                       'status': 'NEW',
                       'stopPrice': '0',
                       'time': 1742255985162,
                       'timeInForce': 'GTC',
                       'type': 'LIMIT',
                    